In [1]:
!pip install implicit
import implicit
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import math

In [2]:
import os
import time

# spark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, explode, desc
from pyspark.sql.types import StringType, ArrayType
from pyspark.mllib.recommendation import ALS
from pyspark.sql.types import IntegerType

# data science imports
import math
import numpy as np
import pandas as pd

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
!pip install findspark
import findspark
findspark.init()

In [4]:
# spark config
spark = SparkSession \
    .builder \
    .appName("meetup recommendation") \
    .getOrCreate()
# get spark contextb
sc = spark.sparkContext

In [5]:
members_rsvp = pd.read_csv("/dbfs/FileStore/tables/members_rsvp.csv")

In [6]:
## Turning the member_id and group_id to category and giving it an idx
members_rsvp['m_code'] = members_rsvp['id'].astype('category').cat.codes
members_rsvp['g_code'] = members_rsvp['group_id'].astype('category').cat.codes

In [7]:
## Get the final dataframe with just the member_codes, group_codes and the scaled rsvps
members_final = members_rsvp[['m_code','g_code','rsvp_total']]

In [8]:
from pyspark.sql import SQLContext
sql = SQLContext(sc)
df_spark = sql.createDataFrame(members_final)

/databricks/spark/python/pyspark/sql/session.py:577: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 arrow_schema = pa.Schema.from_pandas(pdf, preserve_index=False)

In [9]:
type(df_spark)

Out[13]: pyspark.sql.dataframe.DataFrame

In [10]:
df_spark.show()

+------+------+----------+
m_code|g_code|rsvp_total|
+------+------+----------+
 92259| 0| 1.0|
 3776| 0| 1.0|
 8434| 0| 1.0|
230912| 0| 1.0|
 92332| 0| 1.0|
154238| 0| 1.0|
 95258| 0| 1.0|
 19134| 0| 1.0|
 61988| 0| 1.0|
 18304| 0| 1.0|
 28997| 0| 1.0|
 55348| 0| 1.0|
 22034| 0| 1.0|
 49775| 0| 1.0|
116201| 0| 1.0|
 30032| 0| 1.0|
111562| 0| 1.0|
 52612| 0| 1.0|
105885| 0| 1.0|
133391| 0| 1.0|
+------+------+----------+
only showing top 20 rows

In [11]:
ratings_data = df_spark.rdd.map(tuple)

In [12]:
type(ratings_data)

Out[16]: pyspark.rdd.PipelinedRDD

In [13]:
ratings_data.take(3)

Out[17]: [(92259, 0, 1.0), (3776, 0, 1.0), (8434, 0, 1.0)]

In [14]:
## Split data
train, validation, test = ratings_data.randomSplit([6, 2, 2], seed=99)
# cache data
train.cache()
validation.cache()
test.cache()

Out[18]: PythonRDD[9] at RDD at PythonRDD.scala:59

In [15]:
def train_ALS(train_data, validation_data, num_iters, reg_param, ranks):
    """
    Grid Search Function to select the best model based on RMSE of hold-out data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in reg_param:
            # train ALS model
            model = ALS.train(
                ratings=train_data,    # (userID, productID, rating) tuple
                iterations=num_iters,
                rank=rank,
                lambda_=reg,           # regularization param
                seed=99)
            # make prediction
            valid_data = validation_data.map(lambda p: (p[0], p[1]))
            predictions = model.predictAll(valid_data).map(lambda r: ((r[0], r[1]), r[2]))
            # get the rating result
            ratesAndPreds = validation_data.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
            # get the RMSE
            MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
            error = math.sqrt(MSE)
            print('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, error))
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model

In [16]:
# hyper-param config
num_iterations = 10
ranks = [8, 10, 12, 14, 16, 18, 20]
reg_params = [0.001, 0.01, 0.05, 0.1, 0.2]

# grid search and select best model
start_time = time.time()
final_model = train_ALS(train, validation, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

8 latent factors and regularization = 0.001: validation RMSE is 0.7468416484851556
8 latent factors and regularization = 0.01: validation RMSE is 0.4246025821307125
8 latent factors and regularization = 0.05: validation RMSE is 0.332775193302704
8 latent factors and regularization = 0.1: validation RMSE is 0.3249922747366813
8 latent factors and regularization = 0.2: validation RMSE is 0.3573059766626739
10 latent factors and regularization = 0.001: validation RMSE is 0.8446221591267932
10 latent factors and regularization = 0.01: validation RMSE is 0.43212748794731076
10 latent factors and regularization = 0.05: validation RMSE is 0.33063674906863916
10 latent factors and regularization = 0.1: validation RMSE is 0.32463152121921923
10 latent factors and regularization = 0.2: validation RMSE is 0.3573505609470267
12 latent factors and regularization = 0.001: validation RMSE is 0.8474351766142639
12 latent factors and regularization = 0.01: validation RMSE is 0.4399570330239775
12 latent factors and regularization = 0.05: validation RMSE is 0.3287042103618288
12 latent factors and regularization = 0.1: validation RMSE is 0.32352092396215704
12 latent factors and regularization = 0.2: validation RMSE is 0.35725877327261746
14 latent factors and regularization = 0.001: validation RMSE is 0.8871746039676041
14 latent factors and regularization = 0.01: validation RMSE is 0.44996263300657074
14 latent factors and regularization = 0.05: validation RMSE is 0.3292955976932268
14 latent factors and regularization = 0.1: validation RMSE is 0.32404291877526403
14 latent factors and regularization = 0.2: validation RMSE is 0.35753658022500445
16 latent factors and regularization = 0.001: validation RMSE is 0.9975142650291865
16 latent factors and regularization = 0.01: validation RMSE is 0.4615751762798368
16 latent factors and regularization = 0.05: validation RMSE is 0.3280944569901433
16 latent factors and regularization = 0.1: validation RMSE is 0.3236804119454657
16 latent factors and regularization = 0.2: validation RMSE is 0.3573468107241316
18 latent factors and regularization = 0.001: validation RMSE is 0.9359230879470769
18 latent factors and regularization = 0.01: validation RMSE is 0.46372047858840515
18 latent factors and regularization = 0.05: validation RMSE is 0.3284825771574523
18 latent factors and regularization = 0.1: validation RMSE is 0.32412258100210894
18 latent factors and regularization = 0.2: validation RMSE is 0.3575052369626549
20 latent factors and regularization = 0.001: validation RMSE is 0.9654005936616802
20 latent factors and regularization = 0.01: validation RMSE is 0.4723281362092702
20 latent factors and regularization = 0.05: validation RMSE is 0.32647896092785106
20 latent factors and regularization = 0.1: validation RMSE is 0.32314555549724494
20 latent factors and regularization = 0.2: validation RMSE is 0.357079380742376

The best model has 20 latent factors and regularization = 0.1
Total Runtime: 850.90 seconds

In [17]:
## ALS Learning curve
def plot_learning_curve(arr_iters, train_data, validation_data, reg, rank):
    """
    Plot function to show learning curve of ALS
    """
    errors = []
    for num_iters in arr_iters:
        # train ALS model
        model = ALS.train(
            ratings=train_data,    # (userID, productID, rating) tuple
            iterations=num_iters,
            rank=rank,
            lambda_=reg,           # regularization param
            seed=99)
        # make prediction
        valid_data = validation_data.map(lambda p: (p[0], p[1]))
        predictions = model.predictAll(valid_data).map(lambda r: ((r[0], r[1]), r[2]))
        # get the rating result
        ratesAndPreds = validation_data.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
        # get the RMSE
        MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
        error = math.sqrt(MSE)
        # add to errors
        errors.append(error)

    # plot
    plt.figure(figsize=(12, 6))
    plt.plot(arr_iters, errors)
    plt.xlabel('number of iterations')
    plt.ylabel('RMSE')
    plt.title('ALS Learning Curve')
    plt.grid(True)
    plt.display()

In [18]:
# create an array of num_iters
iter_array = list(range(1, 11))
# create learning curve plot
plot_learning_curve(iter_array, train, validation, 0.1, 20)

In [19]:
test_data = test.map(lambda p: (p[0], p[1]))
predictions = final_model.predictAll(test_data).map(lambda r: ((r[0], r[1]), r[2]))
# get the rating result
ratesAndPreds = test.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
# get the RMSE
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
error = math.sqrt(MSE)
print('The out-of-sample RMSE of rating predictions is', round(error, 4))

The out-of-sample RMSE of rating predictions is 0.3262

In [20]:
### Time-Delta implicit recommendation
members_delta = pd.read_csv("/dbfs/FileStore/tables/members_delta.csv")

In [21]:
#turning the member_id and group_id to category and giving it an idx
members_delta['m_code'] = members_delta['id'].astype('category').cat.codes
members_delta['g_code'] = members_delta['group_id'].astype('category').cat.codes

In [22]:
members_delta_final = members_delta[['m_code','g_code','delta']]

In [23]:
members_delta_final.head()

,m_code,g_code,delta
0,122781,0,3.658363
1,5180,0,3.754448
2,11369,0,3.658363
3,296263,0,3.658363
4,122875,0,3.754448


In [24]:
sql = SQLContext(sc)
df_spark_del = sql.createDataFrame(members_delta_final)

In [25]:
type(df_spark_del)

Out[42]: pyspark.sql.dataframe.DataFrame

In [26]:
df_spark_del.show()

+------+------+-----------------+
m_code|g_code| delta|
+------+------+-----------------+
122781| 0|3.658362989323844|
 5180| 0|3.754448398576512|
 11369| 0|3.658362989323844|
296263| 0|3.658362989323844|
122875| 0|3.754448398576512|
201851| 0|3.658362989323844|
126655| 0|3.658362989323844|
 25853| 0|5.067615658362988|
 83143| 0|4.843416370106762|
 24759| 0|4.074733096085409|
 39793| 0|3.850533807829181|
 74564| 0|4.202846975088968|
 30135| 0|3.722419928825623|
 67345| 0|5.195729537366548|
153472| 0|3.658362989323844|
 41181| 0|4.875444839857652|
147382| 0|3.658362989323844|
 71048| 0|3.658362989323844|
140168| 0|5.131672597864768|
175558| 0|3.658362989323844|
+------+------+-----------------+
only showing top 20 rows

In [27]:
delta_data = df_spark_del.rdd.map(tuple)

In [28]:
delta_data.take(3)

Out[45]: [(122781, 0, 3.658362989323844),
 (5180, 0, 3.754448398576512),
 (11369, 0, 3.658362989323844)]

In [29]:
## Split data
train_delta, validation_delta, test_delta = delta_data.randomSplit([6, 2, 2], seed=99)
# cache data
train_delta.cache()
validation_delta.cache()
test_delta.cache()

Out[46]: PythonRDD[11181] at RDD at PythonRDD.scala:59

In [30]:
# hyper-param config
num_iterations = 10
ranks = [8, 10, 12, 14, 16, 18, 20]
reg_params = [0.001, 0.01, 0.05, 0.1, 0.2]

# grid search and select best model
start_time = time.time()
final_model_delta = train_ALS(train_delta, validation_delta, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

8 latent factors and regularization = 0.001: validation RMSE is 2.1665115078772916
8 latent factors and regularization = 0.01: validation RMSE is 1.0896844255087255
8 latent factors and regularization = 0.05: validation RMSE is 0.6151130809943143
8 latent factors and regularization = 0.1: validation RMSE is 0.46066237892427797
8 latent factors and regularization = 0.2: validation RMSE is 0.4413471205700477
10 latent factors and regularization = 0.001: validation RMSE is 2.6479656301815027
10 latent factors and regularization = 0.01: validation RMSE is 1.2409163701485177
10 latent factors and regularization = 0.05: validation RMSE is 0.6378512061328862
10 latent factors and regularization = 0.1: validation RMSE is 0.46045877276113856
10 latent factors and regularization = 0.2: validation RMSE is 0.44206342982678853
12 latent factors and regularization = 0.001: validation RMSE is 2.824029983518413
12 latent factors and regularization = 0.01: validation RMSE is 1.3545826433323789
12 latent factors and regularization = 0.05: validation RMSE is 0.6731871527798061
12 latent factors and regularization = 0.1: validation RMSE is 0.463976085749713
12 latent factors and regularization = 0.2: validation RMSE is 0.44043047185370027
14 latent factors and regularization = 0.001: validation RMSE is 3.100093768877763
14 latent factors and regularization = 0.01: validation RMSE is 1.5026611716285216
14 latent factors and regularization = 0.05: validation RMSE is 0.7066484585484653
14 latent factors and regularization = 0.1: validation RMSE is 0.46364895580957516
14 latent factors and regularization = 0.2: validation RMSE is 0.44081838933161155
16 latent factors and regularization = 0.001: validation RMSE is 3.5123070251365514
16 latent factors and regularization = 0.01: validation RMSE is 1.6364511329285403
16 latent factors and regularization = 0.05: validation RMSE is 0.7253756835030208
16 latent factors and regularization = 0.1: validation RMSE is 0.4669404565937431
16 latent factors and regularization = 0.2: validation RMSE is 0.441846197063241
18 latent factors and regularization = 0.001: validation RMSE is 3.434435790678714
18 latent factors and regularization = 0.01: validation RMSE is 1.717631140357596
18 latent factors and regularization = 0.05: validation RMSE is 0.7303047322899879
18 latent factors and regularization = 0.1: validation RMSE is 0.463329184768621
18 latent factors and regularization = 0.2: validation RMSE is 0.44027911802093994
20 latent factors and regularization = 0.001: validation RMSE is 3.76948747509727
20 latent factors and regularization = 0.01: validation RMSE is 1.815035465643567
20 latent factors and regularization = 0.05: validation RMSE is 0.7500639174776009
20 latent factors and regularization = 0.1: validation RMSE is 0.4642683506112177
20 latent factors and regularization = 0.2: validation RMSE is 0.44029820410537535

The best model has 18 latent factors and regularization = 0.2
Total Runtime: 1122.55 seconds

In [31]:
test_data = test_delta.map(lambda p: (p[0], p[1]))
predictions = final_model_delta.predictAll(test_data).map(lambda r: ((r[0], r[1]), r[2]))
# get the rating result
ratesAndPreds = test_delta.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
# get the RMSE
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
error = math.sqrt(MSE)
print('The out-of-sample RMSE of rating predictions is', round(error, 4))

The out-of-sample RMSE of rating predictions is 0.4382